In [46]:
import pandas as pd
df = pd.read_csv("clean_tweet.csv")
#df.drop(['score','views','comments'],axis=1,inplace=True)
df.head(20)

# https://medium.com/neo4j/machine-learning-on-graphs-fca6eeb8f1d1
# https://markhneedham.com/blog/2018/05/11/node2vec-tensorflow/

,Unnamed: 0,text,postId
0,0,ve always been interested in machine learning ...,5
1,1,as researcher and instructor looking for open ...,7
2,2,not sure if this fits the scope of this se but...,9
3,3,one book that freely available is the elements...,10
4,4,am sure data science as will be discussed in t...,14
5,5,in which situations would one system be prefer...,15
6,6,use href to train data and predict clas,16
7,7,href rel nofollow libsvm is library for suppor,17
8,8,NaN,18
9,9,lots of people use the term em big data em in ...,19


In [47]:
df_p = pd.read_csv("import_code/csvs/tags_posts_rel.csv")
# df.drop(['external_author_id','language','publish_date','harvested_date'],axis=1,inplace=True)
df_p.head(20)

,:START_ID(Post),:END_ID(Tag)
0,5,machine-learning
1,7,education
2,7,open-source
3,14,data-mining
4,14,definitions
5,15,databases
6,16,machine-learning
7,16,bigdata
8,16,libsvm
9,19,bigdata


In [48]:
df_p.columns = ['postId', 'tag']

In [49]:
df_p.head()

,postId,tag
0,5,machine-learning
1,7,education
2,7,open-source
3,14,data-mining
4,14,definitions


In [50]:
my_df = pd.merge(df, df_p, on='postId')

In [51]:
my_df.head(30)

,Unnamed: 0,text,postId,tag
0,0,ve always been interested in machine learning ...,5,machine-learning
1,1,as researcher and instructor looking for open ...,7,education
2,1,as researcher and instructor looking for open ...,7,open-source
3,4,am sure data science as will be discussed in t...,14,data-mining
4,4,am sure data science as will be discussed in t...,14,definitions
5,5,in which situations would one system be prefer...,15,databases
6,6,use href to train data and predict clas,16,machine-learning
7,6,use href to train data and predict clas,16,bigdata
8,6,use href to train data and predict clas,16,libsvm
9,9,lots of people use the term em big data em in ...,19,bigdata


In [52]:
for col in my_df.columns: 
    print(col) 

Unnamed: 0
text
postId
tag


In [53]:
my_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [54]:
my_df.groupby('tag').count()

,text,postId
tag,,
.net,6,6
3d-reconstruction,5,5
ab-test,26,26
accuracy,152,152
activation,1,1
...,...,...
word-embeddings,206,206
word2vec,227,227
xboost,1,1


In [55]:
my_df.head(20)

,text,postId,tag
0,ve always been interested in machine learning ...,5,machine-learning
1,as researcher and instructor looking for open ...,7,education
2,as researcher and instructor looking for open ...,7,open-source
3,am sure data science as will be discussed in t...,14,data-mining
4,am sure data science as will be discussed in t...,14,definitions
5,in which situations would one system be prefer...,15,databases
6,use href to train data and predict clas,16,machine-learning
7,use href to train data and predict clas,16,bigdata
8,use href to train data and predict clas,16,libsvm
9,lots of people use the term em big data em in ...,19,bigdata


In [56]:
my_df.to_csv('tag_and_post.csv',encoding='utf-8')

#### Build graph in neo4j

In [ ]:
from py2neo import Graph, Node, Relationship

graph = Graph(password="password")

In [ ]:
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM "file:////tag_and_post.csv" as row
MERGE (s:TAG{tag:row.tag})
MERGE (t:CONTENT{text:row.text})
CREATE (s)-[:TAGS]->(t)
'''

In [ ]:
# run query
data = graph.run(query)
print(data)

#### all posts with tag 'octave'

In [ ]:
# MATCH p=(t:TAG)-[]->() WHERE t.tag = "octave" RETURN p LIMIT 25

![title](images/tag_octave.png)

#### all shortest path between 'java' and 'python' with some of the edge nodes tags shown

In [ ]:
# MATCH path = allShortestPaths( (t:TAG {tag:"java"})-[*]-(u:TAG {tag:"python"})) RETURN path LIMIT 10;

![title](images/python_java_shortpath.png)

 #### all shortest path between 'education' and 'java'

![title](images/java_education_shortpath.png)

 #### shortest path between 'activation' and 'xboost' with some of the edge nodes tags shown

![title](images/xboost_activation_shortpath.png)

#### Create edge list - node2vec

In [ ]:
# from neo4j.v1 import GraphDatabase, basic_auth

# host = "bolt://127.0.0.1:7687" # replace this with your Sandbox host
# password = "password" # replace this with your Sandbox password

# driver = GraphDatabase.driver(host)

from neo4j import GraphDatabase
import csv

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

# print('Create the edge list')
# with driver.session() as session, open("graph/stack_data.edgelist", "w") as edges_file:
#     result = session.run("""\
#     MATCH (t:TAG)--(other)
#     RETURN id(t) AS source, id(other) AS target
#     """)

#     writer = csv.writer(edges_file, delimiter=" ")

#     for row in result:
#         writer.writerow([row["source"], row["target"]])
        
print('Create the edge list')
with driver.session() as session, open("graph/stack_content.edgelist", "w") as edges_file:
    result = session.run("""\
    MATCH (c:CONTENT)--(other)
    RETURN id(c) AS source, id(other) AS target
    """)

    writer = csv.writer(edges_file, delimiter=" ")

    for row in result:
        writer.writerow([row["source"], row["target"]])

#### node2vec

In [ ]:
# https://github.com/snap-stanford/snap/tree/master/examples/node2vec

# %%bash
# ./node2vec -i:graph/stack_data.edgelist -o:emb/stack_data.emb -l:80 -d:100 -p:0.3 -dr -v

In [ ]:
# (base) Computers-MacBook-Air:node2vec computer$ ./node2vec

# An algorithmic framework for representational learning on graphs. [Dec  2 2019]
# ================================================================================
# usage: node2vec
#    -i:Input graph path (default:'graph/karate.edgelist')
#    -o:Output graph path (default:'emb/karate.emb')
#    -d:Number of dimensions. Default is 128 (default:128)
#    -l:Length of walk per source. Default is 80 (default:80)
#    -r:Number of walks per source. Default is 10 (default:10)
#    -k:Context size for optimization. Default is 10 (default:10)
#    -e:Number of epochs in SGD. Default is 1 (default:1)
#    -p:Return hyperparameter. Default is 1 (default:1)
#    -q:Inout hyperparameter. Default is 1 (default:1)
#    -v Verbose output. 
#    -dr Graph is directed. 
#    -w Graph is weighted. 
#    -ow Output random walks instead of embeddings. 
# (base) Computers-MacBook-Air:node2vec computer$ ./node2vec -i:graph/stack_data.edgelist -o:emb/stack_data.emb -l:80 -d:100 -p:0.3 -dr -v

# An algorithmic framework for representational learning on graphs. [Dec  2 2019]
# ================================================================================
# Input graph path (-i:)=graph/stack_data.edgelist
# Output graph path (-o:)=emb/stack_data.emb
# Number of dimensions. Default is 128 (-d:)=100
# Length of walk per source. Default is 80 (-l:)=80
# Number of walks per source. Default is 10 (-r:)=10
# Context size for optimization. Default is 10 (-k:)=10
# Number of epochs in SGD. Default is 1 (-e:)=1
# Return hyperparameter. Default is 1 (-p:)=0.3
# Inout hyperparameter. Default is 1 (-q:)=1
# Verbose output. (-v)=YES
# Graph is directed. (-dr)=YES
# Graph is weighted. (-w)=NO
# Output random walks instead of embeddings. (-ow)=NO
# Read 53952 lines from graph/stack_data.edgelist
# Preprocessing progress: 99.61% 
# Walking Progress: 95.88%
# Learning Progress: 99.94% 
# (base) Computers-MacBook-Air:node2vec computer$ 

![title](images/node2vec_output.png)

#### Store embeddings in graph

In [28]:
from neo4j import GraphDatabase
import csv

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))


# with open("emb/stack_data.emb", "r") as stack_data_file, driver.session() as session:
#     next(stack_data_file)
#     reader = csv.reader(stack_data_file, delimiter=" ")

#     params = []
#     for row in reader:
#         stack_id = row[0]
#         params.append({
#             "id": int(stack_id),
#             "embedding": [float(item) for item in row[1:]]
#         })

#     session.run("""\
#     UNWIND {params} AS param
#     MATCH (t:Tag) WHERE id(t) = param.id
#     SET t.embedding = param.embedding
#     """, {"params": params})
    
with open("emb/stack_content.emb", "r") as stack_data_file, driver.session() as session:
    next(stack_data_file)
    reader = csv.reader(stack_data_file, delimiter=" ")

    params = []
    for row in reader:
        stack_id = row[0]
        params.append({
            "id": int(stack_id),
            "embedding": [float(item) for item in row[1:]]
        })
        #print(params)

#     session.run("""\
#     UNWIND {params} AS param
#     MATCH (c:CONTENT) WHERE id(c) = param.id
#     SET c.embedding = param.embedding
#     """, {"params": params})
    
    session.run("""\
    UNWIND {params} AS param
    MATCH (t:TAG) WHERE id(t) = param.id
    SET t.embedding = param.embedding
    """, {"params": params})

In [39]:
import pandas as pd

# movies_tag_query = """\
# MATCH (tag:TAG)
# WITH tag ORDER BY tag.name
# WITH collect(id(tag)) AS tags
# MATCH (tag)-[:TAGS]->(c:CONTENT)
# WITH tags, id(c) AS source, c.embedding AS embedding, collect(id(tag)) AS target
# RETURN source, embedding, [t in tags | CASE WHEN t in target THEN 1 ELSE 0 END] AS genres
# """

# with driver.session() as session:
#     result = session.run(movies_tag_query)
#     df = pd.DataFrame([dict(row) for row in result])
    
    
stack_tag_query = """\
MATCH (con:CONTENT)
WITH con ORDER BY con.text
WITH collect(id(con)) AS cons
MATCH (t:TAG)-[:TAGS]->(con)
WITH cons, id(t) AS source, t.embedding AS embedding, collect(id(con)) AS target
RETURN source, embedding, [c in cons | CASE WHEN c in target THEN 1 ELSE 0 END] AS tags
"""

with driver.session() as session:
    result = session.run(stack_tag_query)
    df = pd.DataFrame([dict(row) for row in result])

In [40]:
df.head(40)

,source,embedding,tags
0,0,"[0.336407, -0.330856, 0.441869, 0.0765013, 0.0...","[0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, ..."
1,2,"[0.169663, -0.0214164, 0.0433217, -0.0504316, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4,"[0.174995, -0.0739704, 0.0556046, -0.0417772, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5,"[0.290443, 0.0784903, -0.0502578, -0.0707901, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,7,"[0.182739, -0.043701, 0.0556673, -0.0227042, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,8,"[0.207417, 0.0392582, 0.040614, -0.00937462, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,11,"[0.277072, -0.00572706, -0.0397977, -0.0856387...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
7,12,"[0.151763, -0.0792313, 0.0855298, 0.0131963, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,14,"[0.163391, 0.00748834, 0.0324757, -0.0754839, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,15,"[0.147025, 0.0034152, 0.0563864, -0.0579839, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [66]:
from gensim.models import KeyedVectors

filename = 'emb/stack_content.emb'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
model.most_similar('10799')

[('7048', 0.9953666925430298),
 ('13502', 0.9945465922355652),
 ('11019', 0.9938308000564575),
 ('7522', 0.9936755895614624),
 ('4340', 0.9936710596084595),
 ('4915', 0.9933874607086182),
 ('1971', 0.993089497089386),
 ('2947', 0.9929451942443848),
 ('2026', 0.9929091930389404),
 ('9461', 0.9928749203681946)]

In [67]:
# not working ???

# def neo4j_most_similar(model, key):
#     with driver.session() as session:
#         find_post = "MATCH (t:Tag {id: '%s'}) return id(t)" % key
#         result = session.run(find_post)
#         for r in result:
#             similar_posts = model.most_similar(str(r.value()))
#             for s_post in similar_post:
#                 find_post = "MATCH (t:Tag) where id(m) = %s return m.name" % s_post[0]
#                 similar_post_names = session.run(find_post)
#                 for sm in similar_post_names:
#                     # print the movie name with the cosine similarity
#                     print(sm.value(), s_post[1])
                    
# neo4j_most_similar(model, 'java')

In [68]:
print(neo4j_most_similar(model, '7048'))

None


In [75]:
my_df.head(20)

,text,postId,tag
0,ve always been interested in machine learning ...,5,machine-learning
1,as researcher and instructor looking for open ...,7,education
2,as researcher and instructor looking for open ...,7,open-source
3,am sure data science as will be discussed in t...,14,data-mining
4,am sure data science as will be discussed in t...,14,definitions
5,in which situations would one system be prefer...,15,databases
6,use href to train data and predict clas,16,machine-learning
7,use href to train data and predict clas,16,bigdata
8,use href to train data and predict clas,16,libsvm
9,lots of people use the term em big data em in ...,19,bigdata


In [62]:
print(my_df.loc[[7048], 'text'])

7048    came across href rel nofollow this
Name: text, dtype: object


In [63]:
print(my_df.loc[[10799], 'text'])

10799    lets say we have training data and we have est...
Name: text, dtype: object


In [ ]:
print(my_df.loc[[158], 'tag'])

In [ ]:
print(my_df.loc[[10264], 'tag'])

In [ ]:
print(my_df.loc[[0], 'tag'])

In [ ]:
print(my_df.loc[[23], 'tag'])

In [73]:
model.most_similar('23') 

[('3950', 0.9946283102035522),
 ('12726', 0.9937280416488647),
 ('690', 0.9930843114852905),
 ('185', 0.9927181601524353),
 ('11539', 0.9927140474319458),
 ('15374', 0.9926726222038269),
 ('5151', 0.9926635026931763),
 ('16377', 0.9921455383300781),
 ('8820', 0.9920498132705688),
 ('3814', 0.9920468330383301)]

In [71]:
print(my_df.loc[[4], 'tag'])

4    definitions
Name: tag, dtype: object


In [ ]:
print(my_df.loc[[10799]])

In [ ]:
print(my_df.loc[[7048]])

In [ ]:
print(my_df.loc[[9461]])

In [78]:
model.similarity('17', '15')

0.9184171